# Data Mining Project - ABCDEatsInc.

**Group 16 members:** <br>
- Ana Margarida Valente, 20240936 <br>
- Catarina Carneiro, 20240690 <br>
- Rui Reis, 20240854 <br>
- Mara Mesquita, 20241039 <br>

**MSc:** Data Science and Advanced Analytics - Nova IMS <br>
**Course:** Data Mining <br>
2024/2025

## Introduction
The client, "ABCDEatsInc.", a fictional food delivery service partnering with a range of restaurants to offer diverse meal options, aims to gain a deeper understanding of its customers' behaviors by identifying distinct segments within its database through customer segmentation. A dataset containing data collected over three months from three cities was provided to define, describe, and analyze the resulting clusters. The goal is to uncover actionable insights and briefly recommend marketing strategies tailored to each segment. 

# **Table of Contents** <br>

* [1. Import](#import)
     * [1.1 Import libraries](#libraries)<br>
     * [1.2 Import the dataset](#dataset)<br>
 
* [2. Feature Selection](#select)<br>

* [3. DBSCAN for Outliers](#DBSCAN)<br>
  
* [4. Clustering](#clustering)<br>
    * [4.1 Clustering by perspectives](#perspect)<br>
    * [4.2 Cluster Analysis](#analysis)<br>   

* [5. Profiling with Categorical features & Unused Features ](#profiling)<br>

* [6. Label Outliers](#label)<br>


</div>

<a class="anchor" id="import">

## 1. Import 
    
</a>

<a class="anchor" id="libraries">

## 1.1 Import libraries
    
</a>

In [51]:
#! pip install minisom

In [52]:
import sqlite3
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil

from itertools import product
from scipy.stats import skewnorm

from datetime import datetime
from sklearn.impute import KNNImputer

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

from sklearn.preprocessing import MinMaxScaler, StandardScaler
import joblib
from minisom import MiniSom


from matplotlib.patches import RegularPolygon, Ellipse ## Import Matplotlib functions to create MiniSOM visualizations
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import cm, colorbar
from matplotlib import colors as mpl_colors
from matplotlib.colors import LinearSegmentedColormap

from matplotlib.lines import Line2D
import seaborn as sns

from matplotlib import __version__ as mplver

from sklearn.cluster import MeanShift, DBSCAN, estimate_bandwidth
from sklearn.decomposition import PCA
from collections import Counter
from sklearn.neighbors import NearestNeighbors
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score

from sklearn.base import clone
from sklearn.metrics import pairwise_distances
from scipy.cluster.hierarchy import dendrogram
from sklearn.manifold import TSNE
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split


import matplotlib.cm as cm
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering



# for better resolution plots
%config InlineBackend.figure_format = 'retina' # optionally, you can change 'svg' to 'retina'

# Setting seaborn style
sns.set()

# Display all the df and results
pd.set_option('display.max_rows', None)  
pd.set_option('display.max_columns', None)  
pd.set_option('display.width', 1000)  
pd.set_option('display.colheader_justify', 'center')  

<a class="anchor" id="dataset">

## 1.2 Import the dataset
    
</a>

In [53]:
# Read the CSV file
df = pd.read_csv("data_beforeclustering.csv")

In [54]:
df_backup=df

In [55]:
df=df.set_index("customer_id")

In [ ]:
df.head()

<a class="anchor" id="select">

# 2. Feature Selection
    
</a>

In [ ]:
df.columns.values

In [ ]:
# Splitting feature names into groups
metric_features = ['customer_age', 'vendor_count', 'product_count','first_order', 'last_order',
                   'CUI_American', 'CUI_Asian', 'CUI_Beverages', 'CUI_Cafe','CUI_Chicken Dishes', 
                   'CUI_Chinese', 'CUI_Desserts', 'CUI_Healthy','CUI_Indian', 'CUI_Italian', 
                   'CUI_Japanese', 'CUI_Noodle Dishes','CUI_OTHER', 'CUI_Street Food / Snacks', 'CUI_Thai',
                   'Sunday','Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday',
                   'HR_0', 'HR_1', 'HR_2', 'HR_3', 'HR_4', 'HR_5', 'HR_6', 'HR_7','HR_8', 'HR_9', 'HR_10', 
                   'HR_11', 'HR_12', 'HR_13', 'HR_14','HR_15', 'HR_16', 'HR_17', 'HR_18', 'HR_19', 'HR_20', 'HR_21','HR_22', 'HR_23', 
                   'early_morning(0h-5h)', 'morning(6h-11h)','afternoon(12h-17h)', 'night(18h-23h)',
                   'Sum_of_Orders', 'recency', 'active_period', 'frequency','total_spend', 'cuisine_diversity',
                   'Weekdays','Weekends', 
                   'Main Courses','Snacks and Street Food', 'Desserts and Beverages','Healthy and Special Diets', 'Other'
                   ]

non_metric_features = df.columns[df.columns.str.startswith('enc_')].tolist() 
unused_features = [i for i in df.columns if i not in (metric_features+non_metric_features) ]

# Display
print('metric_features:', metric_features)
print('\nnon_metric_features:', non_metric_features)
print('\nunused_features:', unused_features)

In [ ]:
fig = plt.figure(figsize=(24, 18))  # Increased size for clarity

# Compute Spearman correlation
corr = np.round(df[metric_features].corr(method="spearman"), decimals=2)

# Mask for upper triangle and annotations threshold
mask_annot = np.absolute(corr.values) >= 0.5
annot = np.where(mask_annot, corr.values, np.full(corr.shape, ""))  # Annotate only significant correlations

# Mask upper triangle
matrix = np.triu(np.ones_like(corr, dtype=bool))

# Customize the heatmap
sns.heatmap(
    data=corr,
    annot=annot,                   # Show annotations
    mask=matrix,                   # Apply mask to hide upper triangle
    cmap='PiYG',               # Adjust colormap for better clarity
    fmt='',                        # Avoid formatting issues
    annot_kws={"size": 8},        # Font size for annotations
    vmin=-1, vmax=1, center=0,     # Set limits for the colormap
    square=True, linewidths=.5,    # Keep square cells with a border
    cbar_kws={"shrink": 0.8}       # Shrink color bar for better fit
)

# Title and labels
fig.subplots_adjust(top=0.9)       # Adjust layout to fit the title
fig.suptitle("Correlation Matrix", fontsize=22, weight='bold')  # Bold, larger title
plt.xticks(rotation=45, ha='right', fontsize=12)  # Rotate x-axis labels
plt.yticks(rotation=0, fontsize=12)               # Keep y-axis labels horizontal

# Display the heatmap
plt.show()

Drop from dataset HR, CUI and DOW variables since we have them explicit in other features

- HR - Time periods
- CUI - Types of cuisine
- DOW - Week days and Weekends

In [60]:
# List of columns to drop
columns_to_drop = ['HR_0', 'HR_1', 'HR_2', 'HR_3', 'HR_4', 'HR_5', 'HR_6', 'HR_7', 'HR_8', 'HR_9', 
                   'HR_10', 'HR_11', 'HR_12', 'HR_13', 'HR_14', 'HR_15', 'HR_16', 'HR_17', 'HR_18', 
                   'HR_19', 'HR_20', 'HR_21', 'HR_22', 'HR_23',
                   'Sunday','Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday',
                   'CUI_American', 'CUI_Asian', 'CUI_Beverages', 'CUI_Cafe','CUI_Chicken Dishes', 
                   'CUI_Chinese', 'CUI_Desserts', 'CUI_Healthy','CUI_Indian', 'CUI_Italian', 
                   'CUI_Japanese', 'CUI_Noodle Dishes','CUI_OTHER', 'CUI_Street Food / Snacks', 'CUI_Thai']

metric_features = [i for i in metric_features if i not in columns_to_drop]

df=df.drop(columns=columns_to_drop)

In [ ]:
fig = plt.figure(figsize=(20, 14))  # Increased size for clarity

# Compute Spearman correlation
corr = np.round(df[metric_features].corr(method="spearman"), decimals=2)

# Mask for upper triangle and annotations threshold
mask_annot = np.absolute(corr.values) >= 0.5
annot = np.where(mask_annot, corr.values, np.full(corr.shape, ""))  # Annotate only significant correlations

# Mask upper triangle
matrix = np.triu(np.ones_like(corr, dtype=bool))

# Customize the heatmap
sns.heatmap(
    data=corr,
    annot=annot,                   # Show annotations
    mask=matrix,                   # Apply mask to hide upper triangle
    cmap='PiYG',               # Adjust colormap for better clarity
    fmt='',                        # Avoid formatting issues
    annot_kws={"size": 8},        # Font size for annotations
    vmin=-1, vmax=1, center=0,     # Set limits for the colormap
    square=True, linewidths=.5,    # Keep square cells with a border
    cbar_kws={"shrink": 0.8}       # Shrink color bar for better fit
)

# Title and labels
fig.subplots_adjust(top=0.9)       # Adjust layout to fit the title
fig.suptitle("Correlation Matrix", fontsize=22, weight='bold')  # Bold, larger title
plt.xticks(rotation=45, ha='right', fontsize=12)  # Rotate x-axis labels
plt.yticks(rotation=0, fontsize=12)               # Keep y-axis labels horizontal

# Display the heatmap
plt.show()

## SOM - Visualizing Data

In [ ]:
M = N = 64
neighborhood_function = "gaussian"
topology = "hexagonal"
n_feats = len(metric_features)
learning_rate = 0.7


som_data = df[metric_features].values

sm = MiniSom(M, N,              # 10x10 map size
             n_feats,           # Number of the elements of the vectors in input.
             learning_rate=learning_rate, 
             topology=topology, 
             neighborhood_function=neighborhood_function, 
             activation_distance='euclidean',
             random_seed=42
             )

# Initializes the weights of the SOM picking random samples from data.
sm.random_weights_init(som_data) 


print("Before training:")
print("QE", np.round(sm.quantization_error(som_data),4))
print("TE", np.round(sm.topographic_error(som_data),4))



# Trains the SOM using all the vectors in data sequentially
# minisom does not distinguish between unfolding and fine tuning phase;

sm.train_batch(som_data, 20000)

print("After training:")
print("QE", np.round(sm.quantization_error(som_data),4))
print("TE", np.round(sm.topographic_error(som_data),4))

In [ ]:
weights = sm.get_weights()
weights.shape

In [64]:
def plot_hexagons(som,              # Trained SOM model 
                  sf,               # matplotlib figure object
                  colornorm,        # colornorm
                  matrix_vals,      # SOM weights or
                  label="",         # title for figure
                  cmap=cm.Grays,    # colormap to use
                  annot=False       
                  ):

    
    axs = sf.subplots(1,1)
    
    for i in range(matrix_vals.shape[0]):
        for j in range(matrix_vals.shape[1]):

            wx, wy = som.convert_map_to_euclidean((i,j)) 

            hex = RegularPolygon((wx, wy), 
                                numVertices=6, 
                                radius= np.sqrt(1/3),
                                facecolor=cmap(colornorm(matrix_vals[i, j])), 
                                alpha=1, 
                                edgecolor='white',
                                linewidth=.5)
            axs.add_patch(hex)
            if annot==True:
                annot_val = np.round(matrix_vals[i,j],2)
                if int(annot_val) == annot_val:
                    annot_val = int(annot_val)
                axs.text(wx,wy, annot_val, 
                        ha='center', va='center', 
                        fontsize='x-small')


    ## Remove axes for hex plot
    axs.margins(.05)
    axs.set_aspect('equal')
    axs.axis("off")
    axs.set_title(label)

    

    # ## Add colorbar
    divider = make_axes_locatable(axs)
    ax_cb = divider.append_axes("right", size="5%", pad="0%")

    ## Create a Mappable object
    cmap_sm = plt.cm.ScalarMappable(cmap=cmap, norm=colornorm)
    cmap_sm.set_array([])

    ## Create custom colorbar 
    cb1 = colorbar.Colorbar(ax_cb,
                            orientation='vertical', 
                            alpha=1,
                            mappable=cmap_sm
                            )
    cb1.ax.get_yaxis().labelpad = 6

    # Add colorbar to plot
    sf.add_axes(ax_cb)




    return sf 


In [ ]:
# Plot Component Planes
figsize=(20,17)
fig = plt.figure(figsize=figsize, constrained_layout=True, dpi=128, )

subfigs = fig.subfigures(5,5,wspace=.15)

colornorm = mpl_colors.Normalize(vmin=np.min(weights), vmax=np.max(weights))

for cpi, sf in zip(range(len(metric_features)), subfigs.flatten()):
    
    matrix_vals = weights[:,:,cpi]
    vext = np.max(np.abs([np.min(matrix_vals), np.max(matrix_vals)]))
    colornorm = mpl_colors.Normalize(vmin=np.min(matrix_vals), vmax=np.max(matrix_vals))
    # colornorm = mpl_colors.CenteredNorm(vcenter=0, halfrange=vext)


    sf = plot_hexagons(sm, sf, 
                    colornorm,
                    matrix_vals,
                    label=metric_features[cpi],
                    cmap=cm.coolwarm,
                    )

In [66]:
# Select variables according to their correlations
# Updating metric_features
metric_features.remove('vendor_count') #cuisine_diversity (0.90)
metric_features.remove('product_count') #Sum_of_orders (0.95)
metric_features.remove('last_order') #recency (-1.0)


#Time Periods
metric_features.remove('early_morning(0h-5h)')
metric_features.remove('morning(6h-11h)')
metric_features.remove('afternoon(12h-17h)')
metric_features.remove('night(18h-23h)')

#Type of Meal
metric_features.remove('Main Courses')
metric_features.remove('Snacks and Street Food')
metric_features.remove('Desserts and Beverages')
metric_features.remove('Healthy and Special Diets')
metric_features.remove('Other')


unused_features.extend(['vendor_count','product_count','last_order',
                      'early_morning(0h-5h)','morning(6h-11h)','afternoon(12h-17h)','night(18h-23h)',
                       'Main Courses','Snacks and Street Food','Desserts and Beverages','Healthy and Special Diets','Other'])

In [ ]:
fig = plt.figure(figsize=(24, 18))  # Increased size for clarity

# Compute Spearman correlation
corr = np.round(df[metric_features].corr(method="spearman"), decimals=2)

# Mask for annotations near 0 (e.g., absolute correlation < 0.2)
mask_annot = (np.absolute(corr.values) < 0.2)  # You can adjust the threshold to be closer to 0
annot = np.where(mask_annot, corr.values, np.full(corr.shape, ""))  # Annotate only near-zero correlations

# Mask upper triangle
matrix = np.triu(np.ones_like(corr, dtype=bool))

# Customize the heatmap
sns.heatmap(
    data=corr,
    annot=annot,                   # Show annotations for near-zero correlations
    mask=matrix,                   # Apply mask to hide upper triangle
    cmap='PiYG',                   # Adjust colormap for better clarity
    fmt='',                        # Avoid formatting issues
    annot_kws={"size": 8},         # Font size for annotations
    vmin=-1, vmax=1, center=0,     # Set limits for the colormap
    square=True, linewidths=.5,    # Keep square cells with a border
    cbar_kws={"shrink": 0.8}       # Shrink color bar for better fit
)

# Title and labels
fig.subplots_adjust(top=0.9)       # Adjust layout to fit the title
fig.suptitle("Correlation Matrix (Near Zero)", fontsize=22, weight='bold')  # Bold, larger title
plt.xticks(rotation=45, ha='right', fontsize=12)  # Rotate x-axis labels
plt.yticks(rotation=0, fontsize=12)               # Keep y-axis labels horizontal

# Display the heatmap
plt.show()


**Customer_age** does not seem to have a relevant correlation to any variable

In [68]:
metric_features.remove('customer_age')
unused_features.extend(['customer_age'])

<a class="anchor" id="DBSCAN">

# 3. DBSCAN for Outliers
    
</a>

#### Defining eps and min_samples:
- **MinPts**: As a rule of thumb, **minPts = 2 x dim** can be used.

- **ε**: The value for ε can then be chosen by using a **k-distance graph**, plotting the distance to the kth (k = minPts - 1) nearest neighbor ordered from the largest to the smallest value. Good values of ε are where this plot shows an **"elbow"**


In [ ]:
# Finding dim
dim=len(metric_features)
print(f'dim = {dim}')
print(f'MinPts = {2*dim}')

In [ ]:
# Compute k-distances
n_neighbors = (dim*2)-1  # Number of neighbors to consider (MinPts-1)
neigh = NearestNeighbors(n_neighbors=n_neighbors)
neigh.fit(df[metric_features])
distances, _ = neigh.kneighbors(df[metric_features])

# Sort distances for plotting
distances = np.sort(distances[:, -1])

# Plot k-distance graph
plt.figure(figsize=(8, 6)) 
plt.plot(distances, color='steelblue', linewidth=2, label=f"{n_neighbors}th Nearest Neighbor Distance")

# Add labels and title
plt.title("K-Distance Graph to Determine Optimal Epsilon", fontsize=16, pad=15)
plt.xlabel("Points (Sorted by Distance)", fontsize=14, labelpad=10)
plt.ylabel(f"Distance to {n_neighbors}th Nearest Neighbor", fontsize=14, labelpad=10)

# Add a grid
plt.grid(visible=True, linestyle="--", alpha=0.6)

# Highlight the "elbow" area
plt.axhline(y=1.5, color='red', linestyle="--", linewidth=1.5, label="Potential Elbow Region")
plt.axhline(y=0.5, color='red', linestyle="--", linewidth=1.5)
plt.axhline(y=1, color='green', linestyle="--", linewidth=1.5)

# Customize ticks
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Add legend
plt.legend(fontsize=12, loc="upper left")

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
# Perform DBSCAN clustering
dbscan = DBSCAN(eps=1, min_samples=dim*2, n_jobs=-1)
dbscan_labels = dbscan.fit_predict(df[metric_features])

dbscan_n_clusters = len(np.unique(dbscan_labels))
print("Number of estimated clusters : %d" % dbscan_n_clusters)

In [ ]:
Counter(dbscan_labels)

In [ ]:
# Concatenating the labels to df
df_concat = pd.concat([df[metric_features], pd.Series(dbscan_labels, index=df.index, name="dbscan_labels")], axis=1)
df_concat.head()

In [ ]:
# Detecting noise (potential outliers)
num_noise = len(df_concat.loc[df_concat['dbscan_labels'] == -1])
total_points = len(df_concat)
percent_outliers = (num_noise / total_points) * 100

print(num_noise)
print(f"Percentage of potencial outliers: {percent_outliers:.2f}%")

In [75]:
def get_ss(df):
    """Computes the sum of squares for all variables given a dataset
    """
    ss = np.sum(df.var() * (df.count() - 1))
    return ss  # return sum of sum of squares of each df variable

In [ ]:
# Computing the R^2 of the cluster solution
df_nonoise = df_concat.loc[df_concat['dbscan_labels'] != -1]
sst = get_ss(df[metric_features])  # get total sum of squares
ssw_labels = df_nonoise.groupby(by='dbscan_labels').apply(get_ss)  # compute ssw for each cluster labels
ssb = sst - np.sum(ssw_labels)  # SST = SSW + SSB
r2 = ssb / sst
print("Cluster solution with R^2 of %0.4f" % r2)

In [77]:
# Save the newly detected outliers (they will be classified later based on the final clusters)
df_out = df[dbscan_labels==-1].copy()

# New df without outliers 
df = df[dbscan_labels!=-1].copy()

<a class="anchor" id="clustering">

# 4. Clustering
    
</a>

<a class="anchor" id="perspect">

## 4.1 Clustering by perspectives
    
</a>

In [78]:
# 1. Value-Based Segmentation
value_based_features = ['total_spend', 'frequency', 'recency','active_period']

# 2. Behaviour-Based Segmentation
behaviour_based_features = ['first_order','cuisine_diversity','Weekdays', 'Weekends','Sum_of_Orders']


df_value = df[value_based_features].copy()
df_beh = df[behaviour_based_features].copy()



In [79]:
# Set up the clusterers
kmeans = KMeans(
    init='k-means++',
    n_init=40,
    random_state=42
)

hierarchical = AgglomerativeClustering(
    metric='euclidean'
)

In [80]:
def get_ss(df, feats):
    """
    Calculate the sum of squares (SS) for the given DataFrame.

    The sum of squares is computed as the sum of the variances of each column
    multiplied by the number of non-NA/null observations minus one.

    Parameters:
    df (pandas.DataFrame): The input DataFrame for which the sum of squares is to be calculated.
    feats (list of str): A list of feature column names to be used in the calculation.

    Returns:
    float: The sum of squares of the DataFrame.
    """
    df_ = df[feats]
    ss = np.sum(df_.var() * (df_.count() - 1))
    
    return ss 


def get_ssb(df, feats, label_col):
    """
    Calculate the between-group sum of squares (SSB) for the given DataFrame.
    The between-group sum of squares is computed as the sum of the squared differences
    between the mean of each group and the overall mean, weighted by the number of observations
    in each group.

    Parameters:
    df (pandas.DataFrame): The input DataFrame containing the data.
    feats (list of str): A list of feature column names to be used in the calculation.
    label_col (str): The name of the column in the DataFrame that contains the group labels.
    
    Returns
    float: The between-group sum of squares of the DataFrame.
    """
    
    ssb_i = 0
    for i in np.unique(df[label_col]):
        df_ = df.loc[:, feats]
        X_ = df_.values
        X_k = df_.loc[df[label_col] == i].values
        
        ssb_i += (X_k.shape[0] * (np.square(X_k.mean(axis=0) - X_.mean(axis=0))) )

    ssb = np.sum(ssb_i)
    

    return ssb


def get_ssw(df, feats, label_col):
    """
    Calculate the sum of squared within-cluster distances (SSW) for a given DataFrame.

    Parameters:
    df (pandas.DataFrame): The input DataFrame containing the data.
    feats (list of str): A list of feature column names to be used in the calculation.
    label_col (str): The name of the column containing cluster labels.

    Returns:
    float: The sum of squared within-cluster distances (SSW).
    """
    feats_label = feats+[label_col]

    df_k = df[feats_label].groupby(by=label_col).apply(lambda col: get_ss(col, feats), 
                                                       include_groups=False)

    return df_k.sum()

def get_rsq(df, feats, label_col):
    """
    Calculate the R-squared value for a given DataFrame and features.

    Parameters:
    df (pd.DataFrame): The input DataFrame containing the data.
    feats (list): A list of feature column names to be used in the calculation.
    label_col (str): The name of the column containing the labels or cluster assignments.

    Returns:
    float: The R-squared value, representing the proportion of variance explained by the clustering.
    """

    df_sst_ = get_ss(df, feats)                 # get total sum of squares
    df_ssw_ = get_ssw(df, feats, label_col)     # get ss within
    df_ssb_ = df_sst_ - df_ssw_                 # get ss between

    # r2 = ssb/sst 
    return (df_ssb_/df_sst_)

def get_r2_scores(df, feats, clusterer, min_k=1, max_k=10):
    """
    Loop over different values of k. To be used with sklearn clusterers.
    """
    r2_clust = {}
    for n in range(min_k, max_k):
        clust = clone(clusterer).set_params(n_clusters=n)
        labels = clust.fit_predict(df)
        df_concat = pd.concat([df, 
                               pd.Series(labels, name='labels', index=df.index)], axis=1)  

        r2_clust[n] = get_rsq(df_concat, feats, 'labels' )
    return r2_clust

### 1. Value-Based 

In [ ]:
range_clusters = range(1, 10)
inertia = []
for n_clus in range_clusters:  # iterate over desired ncluster range
    kmclust = KMeans(n_clusters=n_clus, init='k-means++', n_init=40, random_state=1)
    kmclust.fit(df_value)
    inertia.append(kmclust.inertia_)  # save the inertia of the given cluster solution

# The inertia plot
fig, ax = plt.subplots(figsize=(9,5))

ax.plot(range_clusters, inertia)
ax.set_xticks(range_clusters)
ax.set_ylabel("Inertia: SSw")
ax.set_xlabel("Number of clusters")
ax.set_title("Inertia plot over clusters", size=15)

plt.show()

In [ ]:
# Storing average silhouette metric
avg_silhouette = []
for nclus in range_clusters:
    #Skip nclus == 1
    if nclus == 1:
        continue
    
    #Create a figure
    fig = plt.figure(figsize=(13, 7))

    #Initialize the KMeans object with n_clusters value and a random generator
    #seed of 10 for reproducibility.
    kmclust = KMeans(n_clusters=nclus, init='k-means++', n_init=40, random_state=1)
    cluster_labels = kmclust.fit_predict(df_value)

    #The silhouette_score gives the average value for all the samples.
    #This gives a perspective into the density and separation of the formed clusters
    silhouette_avg = silhouette_score(df_value, cluster_labels)
    avg_silhouette.append(silhouette_avg)
    print(f"For n_clusters = {nclus}, the average silhouette_score is : {silhouette_avg}")

    #Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(df_value, cluster_labels)

    y_lower = 10
    for i in range(nclus):
        #Aggregate the silhouette scores for samples belonging to cluster i, and sort them
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]
        ith_cluster_silhouette_values.sort()
        
        #Get y_upper to demarcate silhouette y range size
        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i
        
        #Filling the silhouette
        color = cm.nipy_spectral(float(i) / nclus)
        plt.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        #Label the silhouette plots with their cluster numbers at the middle
        plt.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        #Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    plt.title("The silhouette plot for the various clusters.")
    plt.xlabel("The silhouette coefficient values")
    plt.ylabel("Cluster label")

    #The vertical line for average silhouette score of all the values
    plt.axvline(x=silhouette_avg, color="red", linestyle="--")
    
    #The silhouette coefficient can range from -1, 1
    xmin, xmax = np.round(sample_silhouette_values.min() -0.1, 2), np.round(sample_silhouette_values.max() + 0.1, 2)
    plt.xlim([xmin, xmax])
    
    #The (nclus+1)*10 is for inserting blank space between silhouette
    #plots of individual clusters, to demarcate them clearly.
    plt.ylim([0, len(df_value) + (nclus + 1) * 10])

    plt.yticks([])  # Clear the yaxis labels / ticks
    plt.xticks(np.arange(xmin, xmax, 0.1))

In [83]:
#Obtaining the R² scores for each cluster solution on value based variables
r2_scores = {}

r2_scores['kmeans'] = get_r2_scores(df_value, value_based_features, kmeans)

for linkage in ['complete', 'average', 'single', 'ward']:
    r2_scores[linkage] = get_r2_scores(
        df_value,                 # data
        value_based_features,   # features of perspective
        # use HClust, changing the linkage at each iteration
        hierarchical.set_params(linkage=linkage) 
    )

In [ ]:
r2_scores_df = pd.DataFrame(r2_scores)
r2_scores_df

In [ ]:
# Visualizing the R² scores for each cluster solution on value based variables
r2_scores_df.plot.line(figsize=(10,7))

plt.title("Value Based Variables:\nR² plot for various clustering methods\n", fontsize=21)
plt.legend(title="Cluster methods", title_fontsize=11)
plt.xlabel("Number of clusters", fontsize=13)
plt.ylabel("R² metric", fontsize=13) 
plt.show()      

### 2. Behaviour-Based 

In [ ]:
range_clusters = range(1, 10)
inertia = []
for n_clus in range_clusters:  # iterate over desired ncluster range
    kmclust = KMeans(n_clusters=n_clus, init='k-means++', n_init=40, random_state=1)
    kmclust.fit(df_beh)
    inertia.append(kmclust.inertia_)  # save the inertia of the given cluster solution

# The inertia plot

fig, ax = plt.subplots(figsize=(9,5))

ax.plot(range_clusters, inertia)
ax.set_xticks(range_clusters)
ax.set_ylabel("Inertia: SSw")
ax.set_xlabel("Number of clusters")
ax.set_title("Inertia plot over clusters", size=15)

plt.show()

In [ ]:
# Storing average silhouette metric
avg_silhouette = []
for nclus in range_clusters:
    #Skip nclus == 1
    if nclus == 1:
        continue
    
    #Create a figure
    fig = plt.figure(figsize=(13, 7))

    #Initialize the KMeans object with n_clusters value and a random generator
    #seed of 10 for reproducibility.
    kmclust = KMeans(n_clusters=nclus, init='k-means++', n_init=40, random_state=1)
    cluster_labels = kmclust.fit_predict(df_value)

    #The silhouette_score gives the average value for all the samples.
    #This gives a perspective into the density and separation of the formed clusters
    silhouette_avg = silhouette_score(df_value, cluster_labels)
    avg_silhouette.append(silhouette_avg)
    print(f"For n_clusters = {nclus}, the average silhouette_score is : {silhouette_avg}")

    #Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(df_value, cluster_labels)

    y_lower = 10
    for i in range(nclus):
        #Aggregate the silhouette scores for samples belonging to cluster i, and sort them
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]
        ith_cluster_silhouette_values.sort()
        
        #Get y_upper to demarcate silhouette y range size
        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i
        
        #Filling the silhouette
        color = cm.nipy_spectral(float(i) / nclus)
        plt.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        #Label the silhouette plots with their cluster numbers at the middle
        plt.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        #Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    plt.title("The silhouette plot for the various clusters.")
    plt.xlabel("The silhouette coefficient values")
    plt.ylabel("Cluster label")

    #The vertical line for average silhouette score of all the values
    plt.axvline(x=silhouette_avg, color="red", linestyle="--")
    
    #The silhouette coefficient can range from -1, 1
    xmin, xmax = np.round(sample_silhouette_values.min() -0.1, 2), np.round(sample_silhouette_values.max() + 0.1, 2)
    plt.xlim([xmin, xmax])
    
    #The (nclus+1)*10 is for inserting blank space between silhouette
    #plots of individual clusters, to demarcate them clearly.
    plt.ylim([0, len(df_value) + (nclus + 1) * 10])

    plt.yticks([])  # Clear the yaxis labels / ticks
    plt.xticks(np.arange(xmin, xmax, 0.1))

In [88]:
# Obtaining the R² scores for each cluster solution on Behaviour based variables
r2_scores_beh = {}

r2_scores_beh['kmeans'] = get_r2_scores(df_beh,behaviour_based_features, kmeans)

for linkage in ['complete', 'average', 'single', 'ward']:
    r2_scores_beh[linkage] = get_r2_scores(
        df_beh,                 # data
        behaviour_based_features,   # features of perspective
        # use HClust, changing the linkage at each iteration
        hierarchical.set_params(linkage=linkage) 
    ) 

In [ ]:
r2_scores_df_beh = pd.DataFrame(r2_scores_beh)
r2_scores_df_beh

In [ ]:
# Visualizing the R² scores for each cluster solution on Behaviour based variables
r2_scores_df_beh.plot.line(figsize=(10,7))

plt.title("Preference Based:\nR² plot for various clustering methods\n", fontsize=21)
plt.legend(title="Cluster methods", title_fontsize=11)
plt.xlabel("Number of clusters", fontsize=13)
plt.ylabel("R² metric", fontsize=13)
plt.show()

### 3. Merging the Perspectives

In [91]:
# Applying the right clustering (algorithm and number of clusters) for each perspective
kmeans_value = KMeans(
    n_clusters=2,
    init='k-means++',
    n_init=40,
    random_state=42
)
value_labels = kmeans_value.fit_predict(df_value)

kmeans_beh = KMeans(
    n_clusters=3,
    init='k-means++',
    n_init=40,
    random_state=42
)
behaviour_labels = kmeans_beh.fit_predict(df_beh)


df['value_labels'] = value_labels
df['behaviour_labels'] = behaviour_labels



In [ ]:
# Silhouette score for value-based clustering
silhouette_value = silhouette_score(df_value, value_labels)
print(f"Silhouette Score (Value Clustering): {silhouette_value:.2f}")

# Silhouette score for behavior-based clustering
silhouette_beh = silhouette_score(df_beh, behaviour_labels)
print(f"Silhouette Score (Behavior Clustering): {silhouette_beh:.2f}")


In [ ]:
# Count label frequencies (contigency table)

pd.crosstab(df['value_labels'],
            df['behaviour_labels'])

### 3.1 Manual merging

In [ ]:
# Get centroids of clusters based on value, preference, and behavior labels
df_centroids = df.groupby(['value_labels', 'behaviour_labels'])[metric_features].mean()

df_centroids


In [95]:
# Clusters with low frequency to be merged (<5000):
# (Value_label, Behaviour_label)
to_merge = [(0,1),(0,2),(1,2)]

In [ ]:
# Computing the euclidean distance matrix between the centroids
centroid_dists = euclidean = pairwise_distances(df_centroids)

df_dists = pd.DataFrame(
    centroid_dists, 
    columns=df_centroids.index, 
    index=df_centroids.index
)

df_dists

In [ ]:
# Merging each low frequency clustering (source) 
# to the closest cluster (target)

source_target = {}

for clus in to_merge:
    # If cluster to merge (source) has not yet been used as target
    if clus not in source_target.values():
        # Add this cluster to source_target map as key
        # Use the cluster with the smallest distance to it as value
        source_target[clus] = df_dists.loc[clus].sort_values().index[1]

source_target


In [98]:
df_manual = df.copy()

# Changing the Value_labels and Behaviour_labels based on source_target
for source, target in source_target.items():
    mask = (df_manual['value_labels']==source[0]) & (df_manual['behaviour_labels']==source[1])
    df_manual.loc[mask, 'value_labels'] = target[0]
    df_manual.loc[mask, 'behaviour_labels'] = target[1]

In [ ]:
# New contigency table

pd.crosstab(df_manual['value_labels'],
            df_manual['behaviour_labels'])

In [100]:
# Create a new column combining 'value_labels' and 'behaviour_labels'
df_manual['merged_labels_'] = df_manual['value_labels'].astype(str) + "_" + df_manual['behaviour_labels'].astype(str)

# Use factorize to assign a unique number to each unique combination in 'merged_labels'
df_manual['merged_labels'] = pd.factorize(df_manual['merged_labels_'])[0]

# Drop the string 'merged_labels' column if not needed
df_manual = df_manual.drop(columns=['merged_labels_'])


In [ ]:
# Merged cluster centroids
df_manual.groupby('merged_labels').mean(numeric_only=True)[metric_features]

In [ ]:
# Create a frequency table for 'merged_labels'
frequency_table = df_manual['merged_labels'].value_counts().reset_index()

# Rename columns for clarity
frequency_table.columns = ['Merged Label', 'Frequency']

# Sort by merged labels for better readability (optional)
frequency_table = frequency_table.sort_values(by='Merged Label').reset_index(drop=True)

# Display the table
frequency_table

### 3.2 Merging using Hierarchical clustering

In [ ]:
# Centroids of the concatenated cluster labels
df_centroids = df.groupby(['value_labels', 'behaviour_labels'])\
    [metric_features].mean()
df_centroids

In [103]:
# Using Hierarchical clustering to merge the concatenated cluster centroids
linkage = 'ward'
hclust = AgglomerativeClustering(
    linkage=linkage, 
    metric='euclidean', 
    distance_threshold=0, 
    n_clusters=None
)

hclust_labels = hclust.fit_predict(df_centroids)

In [ ]:
# Adapted from:
# https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_dendrogram.html#sphx-glr-auto-examples-cluster-plot-agglomerative-dendrogram-py

# create the counts of samples under each node (number of points being merged)
counts = np.zeros(hclust.children_.shape[0])
n_samples = len(hclust.labels_)

# hclust.children_ contains the observation ids that are being merged together
# At the i-th iteration, children[i][0] and children[i][1] are merged to form node n_samples + i
for i, merge in enumerate(hclust.children_):
    # track the number of observations in the current cluster being formed
    current_count = 0
    for child_idx in merge:
        if child_idx < n_samples:
            # If this is True, then we are merging an observation
            current_count += 1  # leaf node
        else:
            # Otherwise, we are merging a previously formed cluster
            current_count += counts[child_idx - n_samples]
    counts[i] = current_count

# the hclust.children_ is used to indicate the two points/clusters being merged (dendrogram's u-joins)
# the hclust.distances_ indicates the distance between the two points/clusters (height of the u-joins)
# the counts indicate the number of points being merged (dendrogram's x-axis)
linkage_matrix = np.column_stack(
    [hclust.children_, hclust.distances_, counts]
).astype(float)

# Plot the corresponding dendrogram
sns.set()
fig = plt.figure(figsize=(11,5))
# The Dendrogram parameters need to be tuned

y_threshold = 3.5

dendrogram(linkage_matrix, 
           truncate_mode='level', 
           labels=df_centroids.index, p=5, 
           color_threshold=y_threshold, 
           above_threshold_color='k')

plt.hlines(y_threshold, 0, 1000, colors="r", linestyles="dashed")
plt.title(f'Hierarchical Clustering - {linkage.title()}\'s Dendrogram', fontsize=21)
plt.xlabel('Number of points in node (or index of point if no parenthesis)')
plt.ylabel(f'Euclidean Distance', fontsize=13)
plt.show()

In [ ]:
# Re-running the Hierarchical clustering based on the correct number of clusters
hclust = AgglomerativeClustering(
    linkage='ward', 
    metric='euclidean', 
    n_clusters=3
)
hclust_labels = hclust.fit_predict(df_centroids)
df_centroids['hclust_labels'] = hclust_labels

df_centroids  # centroid's cluster labels

In [ ]:
# Mapper between concatenated clusters and hierarchical clusters
cluster_mapper = df_centroids['hclust_labels'].to_dict()
cluster_mapper

In [ ]:
df_ = df.copy()

# Mapping the hierarchical clusters on the centroids to the observations
df_['merged_labels'] = df_.apply(
    lambda row: cluster_mapper[
        (row['value_labels'], row['behaviour_labels'])
    ], axis=1
)

df_.head()

In [ ]:
# Merged cluster centroids
df_.groupby('merged_labels').mean(numeric_only=True)[metric_features]

In [ ]:
# Merge cluster contigency table
# Getting size of each final cluster
df_counts = df_.groupby('merged_labels')\
    .size()\
    .to_frame()

df_counts

In [ ]:
# Getting the product and behavior labels
df_counts = df_counts\
    .rename({v:k for k, v in cluster_mapper.items()})\
    .reset_index()

df_counts

In [ ]:
df_counts['value_labels'] = df_counts['merged_labels'].apply(lambda x: x[0])
df_counts['behaviour_labels'] = df_counts['merged_labels'].apply(lambda x: x[1])


df_counts


In [ ]:
df_counts.pivot(values=0, index='value_labels', columns='behaviour_labels')


Keep Manual Merging:

In [113]:
df = df_manual.copy()

----------------

## 
<a class="anchor" id="analysis">

## 4.2 Cluster Analysis
    
</a>

In [114]:
def cluster_profiles(df, label_columns, figsize, 
                     cmap="tab10",
                     compare_titles=None):
    """
    Pass df with labels columns of one or multiple clustering labels. 
    Then specify this label columns to perform the cluster profile according to them.
    """
    
    if compare_titles == None:
        compare_titles = [""]*len(label_columns)
        
    fig, axes = plt.subplots(nrows=len(label_columns), 
                             ncols=2, 
                             figsize=figsize, 
                             constrained_layout=True,
                             squeeze=False)
    for ax, label, titl in zip(axes, label_columns, compare_titles):
        # Filtering df
        drop_cols = [i for i in label_columns if i!=label]
        dfax = df.drop(drop_cols, axis=1)
        
        # Getting the cluster centroids and counts
        centroids = dfax.groupby(by=label, as_index=False).mean()
        counts = dfax.groupby(by=label, as_index=False).count().iloc[:,[0,1]]
        counts.columns = [label, "counts"]
        
        # Setting Data
        pd.plotting.parallel_coordinates(centroids, 
                                            label, 
                                            color = sns.color_palette(cmap),
                                            ax=ax[0])



        sns.barplot(x=label, 
                    hue=label,
                    y="counts", 
                    data=counts, 
                    ax=ax[1], 
                    palette=sns.color_palette(cmap),
                    legend=False
                    )

        #Setting Layout
        handles, _ = ax[0].get_legend_handles_labels()
        cluster_labels = ["Cluster {}".format(i) for i in range(len(handles))]
        ax[0].annotate(text=titl, xy=(0.95,1.1), xycoords='axes fraction', fontsize=13, fontweight = 'heavy') 
        ax[0].axhline(color="black", linestyle="--")
        ax[0].set_title("Cluster Means - {} Clusters".format(len(handles)), fontsize=13)
        ax[0].set_xticklabels(ax[0].get_xticklabels(), 
                              rotation=40,
                              ha='right'
                              )
        
        ax[0].legend(handles, cluster_labels,
                     loc='center left', bbox_to_anchor=(1, 0.5), title=label
                     ) # Adaptable to number of clusters
        
        ax[1].set_xticks([i for i in range(len(handles))])
        ax[1].set_xticklabels(cluster_labels)
        ax[1].set_xlabel("")
        ax[1].set_ylabel("Absolute Frequency")
        ax[1].set_title("Cluster Sizes - {} Clusters".format(len(handles)), fontsize=13)
        
        
    
    # plt.subplots_adjust(hspace=0.4, top=0.90)
    plt.suptitle("Cluster Simple Profiling", fontsize=23)
    plt.show()

In [ ]:
# Profilling each cluster 
cluster_profiles(
    df = df[metric_features + ['value_labels', 'behaviour_labels', 'merged_labels']], 
    label_columns = ['value_labels', 'behaviour_labels','merged_labels'], 
    figsize = (28, 13), 
    compare_titles = ["Value clustering", "Behaviour clustering", "Merged clusters"]
)

## 
<a class="anchor" id="profiling">

# 5. Profiling with Categorical features & Unused Features 
    
</a>

## City

In [ ]:
df_city = df[['merged_labels',
            'enc_customer_city_2',
            'enc_customer_city_4',
            'enc_customer_city_8']].groupby(['merged_labels']).sum()

df_city

In [ ]:
df[['merged_labels']].groupby(['merged_labels']).value_counts()

In [ ]:
fig, axes = plt.subplots(1, 
                         df['merged_labels'].nunique() + 1, # Add an extra ax for population countplot
                         figsize=(16,4),
                         tight_layout=True,
                        #  sharey=True,
                         )


for i in range(len(axes.flatten())): 
    ax = axes[i]
    if i == 0:
        sns.countplot(df, 
                        x='customer_city', 
                        order = df['customer_city'].value_counts().index,
                        ax=ax)
        ax.set_title("All Data")
        
    else:    
        sns.countplot(df.loc[df['merged_labels']==i-1], 
                    x='customer_city', 
                    order = df['customer_city'].value_counts().index,
                    ax=ax)
        ax.set_title("Cluster {}".format(i-1))
    
    ax.tick_params(axis="x", labelrotation=90)
    ax.set_xlabel("")
    ax.set_ylabel("")

plt.suptitle("City Counts", )
plt.show()

In [ ]:
df_cl_city = df.groupby([
    "merged_labels", 
    "customer_city",
    ])['customer_city'].size().unstack()

df_cl_city


df_cl_city.plot.bar(stacked=True)

In [ ]:
df_cl_city_pct = df_cl_city.copy()
for i in df['customer_city'].unique():
    df_cl_city_pct[i] = 100*df_cl_city_pct[i]/df['merged_labels'].value_counts().sort_index().values

df_cl_city_pct.plot.bar(stacked=True)

In [ ]:
fig, axes = plt.subplots(1, df['merged_labels'].nunique(), 
                         figsize=(12,4),
                         sharey=True,)

for ax, clust in zip(axes.flatten(), range(df['merged_labels'].nunique())): 
    df_cl = df.loc[df['merged_labels']==clust]
    sns.countplot(df_cl, 
                  x='customer_city', 
                  order = df['customer_city'].value_counts().index,
                  ax=ax)
    
    ax.tick_params(axis="x", labelrotation=90)
    ax.set_xlabel("")

    ax.set_title("Cluster {}".format(clust))
plt.suptitle("Count of City by Cluster", y=1)
plt.show()

In [ ]:
pd.crosstab(df["merged_labels"],df["customer_city"])

## Promo

In [ ]:
df_promo = df[['merged_labels',
            'enc_last_promo',
            ]].groupby(['merged_labels']).sum()

df_promo

In [ ]:
df[['merged_labels']].groupby(['merged_labels']).value_counts()

In [ ]:
fig, axes = plt.subplots(1, 
                         df['merged_labels'].nunique() + 1, # Add an extra ax for population countplot
                         figsize=(16,4),
                         tight_layout=True,
                        #  sharey=True,
                         )


for i in range(len(axes.flatten())): 
    ax = axes[i]
    if i == 0:
        sns.countplot(df, 
                        x='last_promo', 
                        order = df['last_promo'].value_counts().index,
                        ax=ax)
        ax.set_title("All Data")
        
    else:    
        sns.countplot(df.loc[df['merged_labels']==i-1], 
                    x='last_promo', 
                    order = df['last_promo'].value_counts().index,
                    ax=ax)
        ax.set_title("Cluster {}".format(i-1))
    
    ax.tick_params(axis="x", labelrotation=90)
    ax.set_xlabel("")
    ax.set_ylabel("")

plt.suptitle("Promo Counts", )
plt.show()

In [ ]:
df_cl_promo = df.groupby([
    "merged_labels", 
    "last_promo",
    ])['last_promo'].size().unstack()

df_cl_promo


df_cl_promo.plot.bar(stacked=True)

In [ ]:
df_cl_promo_pct = df_cl_promo.copy()
for i in df['last_promo'].unique():
    df_cl_promo_pct[i] = 100*df_cl_promo_pct[i]/df['merged_labels'].value_counts().sort_index().values

df_cl_promo_pct.plot.bar(stacked=True)

In [ ]:
fig, axes = plt.subplots(1, df['merged_labels'].nunique(), 
                         figsize=(12,4),
                         sharey=True,)

for ax, clust in zip(axes.flatten(), range(df['merged_labels'].nunique())): 
    df_cl = df.loc[df['merged_labels']==clust]
    sns.countplot(df_cl, 
                  x='last_promo', 
                  order = df['last_promo'].value_counts().index,
                  ax=ax)
    
    ax.tick_params(axis="x", labelrotation=90)
    ax.set_xlabel("")

    ax.set_title("Cluster {}".format(clust))
plt.suptitle("Count of Last Promo by Cluster", y=1)
plt.show()

## Chain

In [ ]:
df_enc = df[['merged_labels',
            'enc_is_chain_0',
            'enc_is_chain_1',
]].groupby(['merged_labels']).sum()

df_enc

In [ ]:
df[['merged_labels']].groupby(['merged_labels']).value_counts()

In [ ]:
fig, axes = plt.subplots(1, 
                         df['merged_labels'].nunique() + 1,  # Add an extra ax for population countplot
                         figsize=(16, 4),
                         tight_layout=True,
                         )

for i in range(len(axes.flatten())): 
    ax = axes[i]
    if i == 0:
        sns.countplot(df, 
                      x='is_chain', 
                      order=df['is_chain'].value_counts().index,
                      ax=ax)
        ax.set_title("All Data")
        
    else:    
        sns.countplot(df.loc[df['merged_labels'] == i-1],  # Filter by 'merged_labels', not 'is_chain'
                      x='is_chain', 
                      order=df['is_chain'].value_counts().index,
                      ax=ax)
        ax.set_title(f"Cluster {i-1}")
    
    ax.tick_params(axis="x", labelrotation=90)
    ax.set_xlabel("")
    ax.set_ylabel("")

plt.suptitle("Chain Counts", y=1.05)
plt.show()


In [ ]:
df_cl_chain = df.groupby([
    "merged_labels", 
    "is_chain",
    ])['is_chain'].size().unstack()

df_cl_chain


df_cl_chain.plot.bar(stacked=True)

In [ ]:
df_cl_chain_pct = df_cl_chain.copy()
for i in df['is_chain'].unique():
    df_cl_chain_pct[i] = 100*df_cl_chain_pct[i]/df['merged_labels'].value_counts().sort_index().values

df_cl_chain_pct.plot.bar(stacked=True)

In [ ]:
fig, axes = plt.subplots(1, df['merged_labels'].nunique(), 
                         figsize=(12,4),
                         sharey=True,)

for ax, clust in zip(axes.flatten(), range(df['merged_labels'].nunique())): 
    df_cl = df.loc[df['merged_labels']==clust]
    sns.countplot(df_cl, 
                  x='is_chain', 
                  order = df['is_chain'].value_counts().index,
                  ax=ax)
    
    ax.tick_params(axis="x", labelrotation=90)
    ax.set_xlabel("")

    ax.set_title("Cluster {}".format(clust))
plt.suptitle("Count of 'is_chain' by Cluster", y=1)
plt.show()

## Payment Method

In [ ]:
df_pay = df[['merged_labels',
            'enc_payment_method_CARD',
       'enc_payment_method_CASH', 'enc_payment_method_DIGI',
]].groupby(['merged_labels']).sum()

df_pay

In [ ]:
df[['merged_labels']].groupby(['merged_labels']).value_counts()

In [ ]:
fig, axes = plt.subplots(1, 
                         df['merged_labels'].nunique() + 1,  # Add an extra ax for population countplot
                         figsize=(16, 4),
                         tight_layout=True,
                         )

for i in range(len(axes.flatten())): 
    ax = axes[i]
    if i == 0:
        # Plot for all data
        sns.countplot(df, 
                      x='payment_method', 
                      order=df['payment_method'].value_counts().index,
                      ax=ax)
        ax.set_title("All Data")
    else:    
        # Plot for specific clusters
        sns.countplot(df.loc[df['merged_labels'] == i-1],  # Filter by 'merged_labels'
                      x='payment_method', 
                      order=df['payment_method'].value_counts().index,
                      ax=ax)
        ax.set_title(f"Cluster {i-1}")
    
    ax.tick_params(axis="x", labelrotation=90)
    ax.set_xlabel("")
    ax.set_ylabel("")

plt.suptitle("Payment Counts", y=1.05)
plt.show()


In [ ]:
df_cl_pay = df.groupby([
    "merged_labels", 
    "payment_method",
    ])['payment_method'].size().unstack()

df_cl_pay


df_cl_pay.plot.bar(stacked=True)

In [ ]:
df_cl_pay_pct = df_cl_pay.copy()
for i in df['payment_method'].unique():
    df_cl_pay_pct[i] = 100*df_cl_pay_pct[i]/df['merged_labels'].value_counts().sort_index().values

df_cl_pay_pct.plot.bar(stacked=True)

In [ ]:
fig, axes = plt.subplots(1, df['merged_labels'].nunique(), 
                         figsize=(12,4),
                         sharey=True,)

for ax, clust in zip(axes.flatten(), range(df['merged_labels'].nunique())): 
    df_cl = df.loc[df['merged_labels']==clust]
    sns.countplot(df_cl, 
                  x='payment_method', 
                  order = df['payment_method'].value_counts().index,
                  ax=ax)
    
    ax.tick_params(axis="x", labelrotation=90)
    ax.set_xlabel("")

    ax.set_title("Cluster {}".format(clust))
plt.suptitle("Count of The Payment Method by Cluster", y=1)
plt.show()

## Age Group

In [ ]:
df_age = df[['merged_labels',
            'enc_age_group',
       
]].groupby(['merged_labels']).sum()

df_age

In [ ]:
df[['merged_labels']].groupby(['merged_labels']).value_counts()

In [ ]:
fig, axes = plt.subplots(1, 
                         df['merged_labels'].nunique() + 1, # Add an extra ax for population countplot
                         figsize=(16,4),
                         tight_layout=True,
                        #  sharey=True,
                         )


for i in range(len(axes.flatten())): 
    ax = axes[i]
    if i == 0:
        sns.countplot(df, 
                        x='age_group', 
                        order = df['age_group'].value_counts().index,
                        ax=ax)
        ax.set_title("All Data")
        
    else:    
        sns.countplot(df.loc[df['merged_labels']==i-1], 
                    x='age_group', 
                    order = df['age_group'].value_counts().index,
                    ax=ax)
        ax.set_title("Cluster {}".format(i-1))
    
    ax.tick_params(axis="x", labelrotation=90)
    ax.set_xlabel("")
    ax.set_ylabel("")

plt.suptitle("Age Group Counts", )
plt.show()

In [ ]:
df_cl_age = df.groupby([
    "merged_labels", 
    "age_group",
    ])['age_group'].size().unstack()

df_cl_age


df_cl_age.plot.bar(stacked=True)

In [ ]:
df_cl_age_pct = df_cl_age.copy()
for i in df['age_group'].unique():
    df_cl_age_pct[i] = 100*df_cl_age_pct[i]/df['merged_labels'].value_counts().sort_index().values

df_cl_age_pct.plot.bar(stacked=True)

In [ ]:
fig, axes = plt.subplots(1, df['merged_labels'].nunique(), 
                         figsize=(12,4),
                         sharey=True,)

for ax, clust in zip(axes.flatten(), range(df['merged_labels'].nunique())): 
    df_cl = df.loc[df['merged_labels']==clust]
    sns.countplot(df_cl, 
                  x='age_group', 
                  order = df['age_group'].value_counts().index,
                  ax=ax)
    
    ax.tick_params(axis="x", labelrotation=90)
    ax.set_xlabel("")

    ax.set_title("Cluster {}".format(clust))
plt.suptitle("Count of The Age Group by Cluster", y=1)
plt.show()

## Cuisine Types

In [144]:
# Load the previously saved scaler
scaler = joblib.load('scaler.pkl')

num_feats = scaler.keys()

In [145]:
cat_feats = joblib.load('cat_feats.pkl')

cat_feats += ['value_labels', 'behaviour_labels', 'merged_labels']

In [ ]:
#get all features in df columns and the list num_feats

# variables related to spending are:
spending_cols = [
    'Main Courses',
    'Snacks and Street Food',
    'Desserts and Beverages',
    'Healthy and Special Diets',
    'Other'
]

cols_to_scale = list(set(df.columns).intersection(num_feats))

original_val = df.copy()

for col in cols_to_scale:
   original_val[col] = scaler[col].inverse_transform(pd.DataFrame (df[col]))


threshold = 1e-5
for col in cols_to_scale:
   original_val[col] = original_val[col].apply(lambda x: 0 if abs(x) < threshold else x)


# Replace the scaled columns in the original DataFrame
df[spending_cols] = original_val[spending_cols]

# Calculate the cluster profile with the original spending values
cluster_profile_original = df.groupby('merged_labels')[spending_cols].agg(['mean', 'std', 'min', 'max', 'median'])



# Display the cluster profile with the original spending values
cluster_profile_original.head()



In [ ]:
# Get the mean values for the non-scaled spending columns
cluster_means_original = cluster_profile_original.xs('mean', axis=1, level=1)

# Plot the grouped bar chart for the original (non-scaled) values
cluster_means_original.T.plot(kind='bar', figsize=(10, 6), colormap='tab10', edgecolor='black')
plt.title('Average Spending by Cuisine Type in Each Cluster (Original Values)')
plt.xlabel('Cuisine Types')
plt.ylabel('Average Spending')
plt.legend(title='Cluster', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45)
plt.show()





## Time Period

In [ ]:
# variables related to time are:
time_cols = [
'early_morning(0h-5h)',
 'morning(6h-11h)',
 'afternoon(12h-17h)',
 'night(18h-23h)'
]


# Replace the scaled columns in the original DataFrame
df[time_cols] = original_val[time_cols]

# Calculate the cluster profile with the original time values
cluster_profile_original_t = df.groupby('merged_labels')[time_cols].agg(['mean', 'std', 'min', 'max', 'median'])

# Display the cluster profile with the original time values
cluster_profile_original_t.head()

In [ ]:
# Calculate means
cluster_means_time = df.groupby('merged_labels')[time_cols].mean()

# Plot the grouped bar chart
cluster_means_time.T.plot(kind='bar', figsize=(10, 6), colormap='tab10', edgecolor='black')
plt.title('Average Orders in the Time period in Each Cluster')
plt.xlabel('Time Periods')
plt.ylabel('Average Orders')
plt.legend(title='Cluster', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45)
plt.show()


-------------------------

In [ ]:
two_dim = TSNE(random_state=42).fit_transform(df[metric_features])

pd.DataFrame(two_dim).plot.scatter(x=0, y=1, c=df['merged_labels'], colormap='tab10', figsize=(15,10))
plt.show()


## 
<a class="anchor" id="label">

# 6. Label Outliers
</a>

In [151]:
def get_ss_variables(df):
    """Get the SS for each variable
    """
    ss_vars = df.var() * (df.count() - 1)
    return ss_vars

def r2_variables(df, labels):
    """Get the R² for each variable
    """
    sst_vars = get_ss_variables(df)
    ssw_vars = np.sum(df.groupby(labels).apply(get_ss_variables))
    return 1 - ssw_vars/sst_vars

In [ ]:
# We are essentially decomposing the R² into the R² for each variable
r2_variables(df[metric_features + ['merged_labels']], 'merged_labels').drop('merged_labels')

In [ ]:
# Preparing the data
X = df[metric_features]
y = df.merged_labels

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Fitting the decision tree
dt = DecisionTreeClassifier(random_state=42, max_depth=3)
dt.fit(X_train, y_train)
print("It is estimated that in average, we are able to predict {0:.2f}% of the customers correctly".format(dt.score(X_test, y_test)*100))


# Performance on training data
train_score = dt.score(X_train, y_train)
# Performance on test data
test_score = dt.score(X_test, y_test)

print(f"Training Accuracy: {train_score * 100:.2f}%")
print(f"Test Accuracy: {test_score * 100:.2f}%")


In [ ]:
# Assessing feature importance
pd.Series(dt.feature_importances_, index=X_train.columns).sort_values(ascending=False)

In [ ]:
# Predicting the cluster labels of the outliers
df_out['merged_labels'] = dt.predict(df_out[metric_features])
df_out.head()

In [156]:
# Concatenate df_out with the predicted 'merged_labels' back to the original df
df = pd.concat([df, df_out[['merged_labels']]], axis=0)
